# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../outputData/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,queenstown,-45.0322,168.6610,5.26,95,100,3.92,NZ,1694985168
1,1,tadine,-21.5483,167.8823,21.72,85,48,7.79,NC,1694985169
2,2,chonchi,-42.6240,-73.7724,7.79,71,59,1.98,CL,1694985170
3,3,benguela,-12.5790,13.4037,22.21,91,100,0.98,AO,1694985170
4,4,carnarvon,-24.8826,113.6576,17.85,73,50,9.72,AU,1694985171


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
%%capture --no-display

# Configure the map plot
H_gamma = '#2800ff'

# Display the map
city_data_df.hvplot.scatter(x = 'Lng', y = 'Lat', c = H_gamma, s = 'Humidity', width = 800, height = 400 )



:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:

# Drop null values #
city_data_df.dropna()


## Temp between 15 and 30 C, no super crazy humidity, not super windy ##
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= 30) & (city_data_df["Max Temp"] >= 15)\
                & (city_data_df["Humidity"] >= 40) & (city_data_df["Humidity"] <= 80)\
                & (city_data_df["Wind Speed"] <= 4.5),\
                                :]

ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,hawaiian paradise park,19.5850,-154.9731,28.19,78,100,2.06,US,1694985173
7,7,albany,42.6512,-73.7550,22.29,71,75,2.57,US,1694985013
24,24,nova vicosa,-17.8908,-39.3725,23.66,78,2,3.81,BR,1694985187
25,25,kone,-21.0608,164.8619,22.46,73,8,2.16,NC,1694985188
35,35,adam,22.3840,57.5240,28.62,65,99,3.60,OM,1694985195
...,...,...,...,...,...,...,...,...,...,...
520,520,saint-francois,47.0019,-70.8129,23.87,64,44,1.03,CA,1694985561
521,521,bafia,4.7399,11.2206,18.82,74,100,0.75,CM,1694985562
523,523,whakatane,-37.9519,176.9946,18.64,75,90,1.79,NZ,1694985563
524,524,chanika,-1.4482,31.0892,18.36,77,36,1.46,TZ,1694985564


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/hf/mp5v5rls3hg252gfcnygxxpw0000gn/T/ipykernel_13351/2149755058.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
6,hawaiian paradise park,US,19.5850,-154.9731,78,
7,albany,US,42.6512,-73.7550,71,
24,nova vicosa,BR,-17.8908,-39.3725,78,
25,kone,NC,-21.0608,164.8619,73,
35,adam,OM,22.3840,57.5240,65,
...,...,...,...,...,...,...
520,saint-francois,CA,47.0019,-70.8129,64,
521,bafia,CM,4.7399,11.2206,74,
523,whakatane,NZ,-37.9519,176.9946,75,
524,chanika,TZ,-1.4482,31.0892,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
category = "categories=accommodation.hotel"
apiKey = f"&apiKey={geoapify_key}"
limit = "&limit=20"

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    cityLat = row["Lat"]
    cityLong = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f"&filter=circle:{cityLong},{cityLat},{radius}"
    bias = f"&bias=proximity:{cityLong},{cityLat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url+category+filter+bias+limit+apiKey )
    
    # Convert the API response to JSON format
    name_addressJSON = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_addressJSON["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search...
hawaiian paradise park - nearest hotel: No hotel found
albany - nearest hotel: Renaissance Albany Hotel
nova vicosa - nearest hotel: Pousada Pontal da Barra
kone - nearest hotel: L'escale
adam - nearest hotel: No hotel found
essaouira - nearest hotel: Tweets Hotel
soke - nearest hotel: N.Deniz Butik Otel
sebastopol - nearest hotel: Отель Олимп
zharkent - nearest hotel: Hotel (good)
rukban - nearest hotel: No hotel found
zarzis - nearest hotel: Zephir Hotel & Spa
estrela - nearest hotel: Estrela Palace Hotel
slave lake - nearest hotel: Northwest Inn
thompson - nearest hotel: Mystery Lake Hotel
kayangel - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
durban - nearest hotel: Butterworth
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
portland - nearest hotel: Marriott
colonia - nearest hotel: Hotel Am Rathaus
kupang - nearest hotel: Hotel Greenia
san alberto - nearest hotel: Hotel Aguai
ain sukhna - nearest hotel: ا

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hawaiian paradise park,US,19.5850,-154.9731,78,No hotel found
7,albany,US,42.6512,-73.7550,71,Renaissance Albany Hotel
24,nova vicosa,BR,-17.8908,-39.3725,78,Pousada Pontal da Barra
25,kone,NC,-21.0608,164.8619,73,L'escale
35,adam,OM,22.3840,57.5240,65,No hotel found
...,...,...,...,...,...,...
520,saint-francois,CA,47.0019,-70.8129,64,No hotel found
521,bafia,CM,4.7399,11.2206,74,Hôtel NJUKEMTA
523,whakatane,NZ,-37.9519,176.9946,75,Whakatane Hotel
524,chanika,TZ,-1.4482,31.0892,77,Hotel at home


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
%%capture --no-display

h_Alpha = "#ff0000"

# Configure the map plot
hotel_df.hvplot.scatter( x = 'Lng', y = 'Lat', c = h_Alpha, s = 75, alpha = 0.75, width = 800, height = 400, hover_cols = ["Hotel Name", "Country"] )

# Display the map


:Scatter   [Lng]   (Lat,Hotel Name,Country)